<a href="https://colab.research.google.com/github/DiegoFChaves/WordCloud/blob/main/IAFIN_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Máquinas de Vetores de Suporte (SVMs)**

*Prof. Dr. Orlando Junior*

**Support Vector Machines *Texto em itálico* (SVMs)** são modelos de aprendizado supervisionado usados tanto para classificação quanto para regressão, embora sejam mais conhecidos por resolver problemas de classificação. A ideia principal é encontrar um hiperplano que melhor separe os dados em diferentes classes, maximizando a margem entre os pontos de dados mais próximos de cada classe, chamados de vetores de suporte.

## Bibliotecas

In [ ]:
# Bibliotecas para análise e processamento de dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Bibliotecas para amostragem e normalização
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Bibliotecas para aprendizagem
# https://scikit-learn.org/dev/api/sklearn.svm.html
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Bibliotecas para avaliação da aprendizagem
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score

## Dados

In [ ]:
df = pd.read_excel("LIVRARIAS_DORELA_MOD.xls", sheet_name="DADOS")
df.head()

,CLIENTE,STATUS,IDADE,UNIFED,BSTATE_SP,BSTATE_RJ,FONE,BFONE,INSTRU,BINST_PR_SEC,...,RESID,BRESID,ATRASO,FICÇÃO,BFIX,NÃOFICÇAO,BNOFIX,AUTOAJUDA,BAUTAJ,CATEG
0,LDOR_0001,MAU,26,SP,1,0,SIM,1,PRIM & SEC,1,...,PROP,1,41,SIM,1,NAO,0,SIM,1,1
1,LDOR_0002,MAU,43,SP,1,0,SIM,1,SUP,0,...,ALUG,0,40,NAO,0,NAO,0,SIM,1,1
2,LDOR_0003,BOM,33,OUTROS,0,0,SIM,1,SUP,0,...,PROP,1,3,SIM,1,NAO,0,NAO,0,0
3,LDOR_0004,BOM,39,RJ,0,1,SIM,1,MV,0,...,PROP,1,4,SIM,1,NAO,0,NAO,0,1
4,LDOR_0005,BOM,43,OUTROS,0,0,SIM,1,PRIM & SEC,1,...,PROP,1,0,SIM,1,NAO,0,NAO,0,1


In [ ]:
df.info()

## Seleção e Preparação de Dados

In [ ]:
y = df['STATUS'].astype('category')

In [ ]:
X_cat = df[['UNIFED', 'FONE', 'INSTRU', 'RESTR', 'FICÇÃO', 'NÃOFICÇAO', 'AUTOAJUDA']]
X_cat = X_cat.astype('category')
X_cat.head()

,UNIFED,FONE,INSTRU,RESTR,FICÇÃO,NÃOFICÇAO,AUTOAJUDA
0,SP,SIM,PRIM & SEC,SIM,SIM,NAO,SIM
1,SP,SIM,SUP,SIM,NAO,NAO,SIM
2,OUTROS,SIM,SUP,SIM,SIM,NAO,NAO
3,RJ,SIM,MV,NAO,SIM,NAO,NAO
4,OUTROS,SIM,PRIM & SEC,NAO,SIM,NAO,NAO


In [ ]:
X_cat_dummies = pd.get_dummies(X_cat)
X_cat_dummies.head()

,UNIFED_OUTROS,UNIFED_RJ,UNIFED_SP,FONE_NÃO,FONE_SIM,INSTRU_MV,INSTRU_PRIM & SEC,INSTRU_SUP,RESTR_NAO,RESTR_SIM,FICÇÃO_NAO,FICÇÃO_SIM,NÃOFICÇAO_NAO,NÃOFICÇAO_SIM,AUTOAJUDA_NAO,AUTOAJUDA_SIM
0,False,False,True,False,True,False,True,False,False,True,False,True,True,False,False,True
1,False,False,True,False,True,False,False,True,False,True,True,False,True,False,False,True
2,True,False,False,False,True,False,False,True,False,True,False,True,True,False,True,False
3,False,True,False,False,True,True,False,False,True,False,False,True,True,False,True,False
4,True,False,False,False,True,False,True,False,True,False,False,True,True,False,True,False


In [ ]:
X_cat_dummies.info()

In [ ]:
X = X_cat_dummies

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Treinamento: ", len(X_train))
print("Teste      : ", len(X_test))

Treinamento:  2100
Teste      :  900


In [ ]:
X_train.info()

In [ ]:
y_train.value_counts()/sum(y_train.value_counts())

,count
STATUS,
BOM,0.797619
MAU,0.202381


In [ ]:
y_test.value_counts()/sum(y_test.value_counts())

,count
STATUS,
BOM,0.805556
MAU,0.194444


## Treinamento e Avaliação

### LinearSVC

In [ ]:
params = {'C': [0.01, 0.1, 1.0, 10, 100]}

In [ ]:
svm = LinearSVC(class_weight={'BOM': 0.8, 'MAU': 0.2})

In [ ]:
gs = GridSearchCV(estimator=svm, param_grid=params, cv=10, verbose=0)

In [ ]:
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LinearSVC(class_weight={'BOM': 0.8, 'MAU': 0.2}),
             param_grid={'C': [0.01, 0.1, 1.0, 10, 100]})

In [ ]:
y_pred = gs.best_estimator_.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))

[[725   0]
 [175   0]]


In [ ]:
print("C = %.2f" % gs.best_estimator_.C)
print("Acurácia      : %.5f" % accuracy_score(y_test, y_pred))
print("Precisão (BOM): %.5f" % precision_score(y_test, y_pred, pos_label='BOM'))
print("Precisão (MAU): %.5f" % precision_score(y_test, y_pred, pos_label='MAU'))

C = 0.01
Acurácia      : 0.80556
Precisão (BOM): 0.80556
Precisão (MAU): 0.00000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### SVC Poly

In [ ]:
svm_poly = SVC(kernel='poly', degree=3, C=0.01, gamma=1.0, class_weight={'BOM': 0.8, 'MAU': 0.2})

In [ ]:
svm_poly.fit(X_train, y_train)

SVC(C=0.01, class_weight={'BOM': 0.8, 'MAU': 0.2}, gamma=1.0, kernel='poly')

In [ ]:
y_pred_poly = svm_poly.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred_poly))

[[722   3]
 [158  17]]


In [ ]:
print("Acurácia      : %.5f" % accuracy_score(y_test, y_pred_poly))
print("Precisão (BOM): %.5f" % precision_score(y_test, y_pred_poly, pos_label='BOM'))
print("Precisão (MAU): %.5f" % precision_score(y_test, y_pred_poly, pos_label='MAU'))

Acurácia      : 0.82111
Precisão (BOM): 0.82045
Precisão (MAU): 0.85000


### SVC RBF

In [ ]:
svm_rbf = SVC(kernel='rbf', C=1.0, gamma=1.0, class_weight={'BOM': 0.8, 'MAU': 0.2})

In [ ]:
svm_rbf.fit(X_train, y_train)

SVC(class_weight={'BOM': 0.8, 'MAU': 0.2}, gamma=1.0)

In [ ]:
y_pred_rbf = svm_rbf.predict(X_test)

In [ ]:
print("Acurácia      : %.5f" % accuracy_score(y_test, y_pred_rbf))
print("Precisão (BOM): %.5f" % precision_score(y_test, y_pred_rbf, pos_label='BOM'))
print("Precisão (MAU): %.5f" % precision_score(y_test, y_pred_rbf, pos_label='MAU'))

Acurácia      : 0.82333
Precisão (BOM): 0.82232
Precisão (MAU): 0.86364
